In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Apr  2 12:37:54 2020

@author: mor
"""

import multiprocessing
from gensim.models import Word2Vec, FastText
from gensim.models.word2vec import LineSentence
import time
import re
import nltk
import codecs
import string
import subprocess
import unicodedata
import os
from io import StringIO
import pandas as pd
from operator import itemgetter
import random
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument



In [ ]:

BASE = os.getcwd()
DATA = os.path.join(BASE,'data')
MODELS = os.path.join(BASE,'models')



In [ ]:

# =============================================================================
# train Word2Vec, FastText, Doc2Vec
# =============================================================================


In [ ]:


# =============================================================================
# train Word2Vec
# ============================================================================= 
def train(inp = DATA+"/texts.txt",out_model = MODELS+"/word2vec.model"):

    start = time.time()

    model = Word2Vec(LineSentence(inp), sg = 1, # 0=CBOW , 1= SkipGram
                     size=100, window=5, min_count=5, workers=multiprocessing.cpu_count())

    # trim unneeded model memory = use (much) less RAM
    model.init_sims(replace=True)

    print(time.time()-start)

    model.save(out_model)


# =============================================================================
# train FastText
# =============================================================================    
def trainFastText(inp = DATA+"/texts.txt",out_model = MODELS+"/fasttext.model"):

    start = time.time()

    model = FastText(LineSentence(inp) ,sg = 1, # 0=CBOW , 1= SkipGram
                     size=100, window=5, min_count=5, workers=multiprocessing.cpu_count())

    # trim unneeded model memory = use (much) less RAM
    model.init_sims(replace=True)

    print(time.time()-start)

    model.save(out_model)
    



In [ ]:


# =============================================================================
# load models
# =============================================================================
modelWord2Vec =  Word2Vec.load(MODELS+"/word2vec.model")
modelFastText =  FastText.load(MODELS+"/fasttext.model")




In [ ]:

# =============================================================================
# train doc2vec
# ============================================================================= 


In [ ]:


from nltk.tokenize import word_tokenize

df = pd.read_csv(DATA+'/cases.csv')
    
data = df[['id','text']].values
tagged_data = [TaggedDocument(words=word_tokenize(d[1]), tags=[d[0]]) for d in data]



In [ ]:

#train model
max_epochs = 100
vec_size = 200
alpha = 0.025

def train_doc2vec(tagged_data,max_epochs=100,vec_size=200,alpha=0.025,workers=4, path=MODELS+"/doc2vec.model"):

    model = Doc2Vec(vector_size=vec_size,
                    alpha=alpha, 
                    min_alpha=0.00025,
                    min_count=1,
                    dm =1,#'distributed memory' (PV-DM)
                    workers=4)
      
    model.build_vocab(tagged_data)
    
    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(tagged_data,
                    total_examples=model.corpus_count,
                    epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha
    
    model.save(path)
    print("Model Saved")



In [ ]:

# =============================================================================
# load model
# =============================================================================
model = Doc2Vec.load(MODELS+"/doc2vec.model")


